In [92]:
import matplotlib.pyplot as plt
import math
import numpy as np

from mpl_toolkits.mplot3d import Axes3D

from effsim.paramgen import uniform_param_generator
from effsim.paramgen import distribution_param_generator
from effsim.effsim import EventSimulator
from effsim.effsim import NoiseMaker

from pytpc.hdfdata import HDFDataFile

import pytpc
from pytpc.fitting.montecarlo import MCFitter

from pytpc.tpcplot import pad_plot, chamber_plot
from pytpc.constants import degrees, pi
from math import sin, cos
import csv
import yaml
import h5py

In [93]:
# read data from file - paths changed to /home/chen
with open('/home/chen/config_e15503a_runs_92-104.yml', 'r') as f:
    config = yaml.load(f)
    
# Extract parameters from test data
beam_enu0 = config['beam_enu0']
beam_mass = config['beam_mass']
beam_charge = config['beam_charge']
mass_num = config['mass_num']
max_beam_angle = (config['max_beam_angle']*math.pi)/180
beam_origin_z = config['beam_origin_z']
gas = pytpc.gases.InterpolatedGas('isobutane', 19.2)

#z0 = 0.5
# create & observe a single event
num_evts = 1000
#print(config)
pgen = uniform_param_generator(beam_enu0, beam_mass, beam_charge, mass_num, max_beam_angle, beam_origin_z, gas, num_evts)

In [94]:
sim = EventSimulator(config)
p = next(pgen)

In [95]:
evtClean, ctr = sim.make_event(p[0][0], p[0][1], p[0][2], p[0][3], p[0][4], p[0][5])
#print(evtClean)

In [96]:
hits = np.zeros(10240)
for k, v in evtClean.items():
    hits[k] += v.max()

In [97]:
for padAddress in range(0,10240):
    if (padAddress not in evtClean):
        evtClean[padAddress] = np.zeros(512)

In [98]:
pyevtClean = sim.convert_event(evtClean)

In [99]:
xyzs = pyevtClean.xyzs(peaks_only=True, drift_vel=5.2, clock=12.5, return_pads=True, baseline_correction=True, cg_times=True)

/home/chen/anaconda3/lib/python3.6/site-packages/pytpc-1.1.0-py3.6-linux-x86_64.egg/pytpc/evtdata.py:187: RuntimeWarning: divide by zero encountered in true_divide
  return np.sum(peak_data * range_idx, axis=-1) / peak_data.sum(axis=-1)
/home/chen/anaconda3/lib/python3.6/site-packages/pytpc-1.1.0-py3.6-linux-x86_64.egg/pytpc/evtdata.py:187: RuntimeWarning: invalid value encountered in true_divide
  return np.sum(peak_data * range_idx, axis=-1) / peak_data.sum(axis=-1)


In [100]:
print(xyzs) # format: [x, y, z, a, pad, ...]



[[ 1.03072938e+02  1.42027211e+01  1.90301091e+03  6.00000000e+00
   5.30000000e+02]
 [ 1.05527056e+02  1.55518732e+01  1.89922909e+03  6.00000000e+00
   5.32000000e+02]
 [ 1.10435291e+02  1.55518732e+01             nan  0.00000000e+00
   5.37000000e+02]
 ...
 [ 4.90823515e+01 -2.66408330e+02             nan  0.00000000e+00
   1.02370000e+04]
 [ 5.39905866e+01 -2.69377567e+02             nan  0.00000000e+00
   1.02380000e+04]
 [ 5.88988218e+01 -2.66408330e+02             nan  0.00000000e+00
   1.02390000e+04]]


In [101]:
tilt = config['tilt']*degrees
#print(tilt)
trans3D = np.array([[1, 0, 0],
                  [0, cos(-tilt), -sin(-tilt)],
                  [0, sin(-tilt), cos(-tilt)]])
trans2D = trans = np.array([[cos(-tilt), -sin(-tilt)],
                  [sin(-tilt), cos(-tilt)]])

In [102]:
for i in xyzs:
    xyz_coordinate = np.array([i[0],i[1],i[2]])
    uvw_coordinate = np.dot(trans3D, xyz_coordinate)
    [i[0],i[1],i[2]] = uvw_coordinate

print(xyzs)

[[1.03072938e+02 2.19643601e+02 1.89034624e+03 6.00000000e+00
  5.30000000e+02]
 [1.05527056e+02 2.20576428e+02 1.88644084e+03 6.00000000e+00
  5.32000000e+02]
 [           nan            nan            nan 0.00000000e+00
  5.37000000e+02]
 ...
 [           nan            nan            nan 0.00000000e+00
  1.02370000e+04]
 [           nan            nan            nan 0.00000000e+00
  1.02380000e+04]
 [           nan            nan            nan 0.00000000e+00
  1.02390000e+04]]


In [103]:
uvw_center = np.dot(trans2D, ctr)
cu = float(uvw_center[0])
cv = float(uvw_center[1])

In [104]:
#xyz = dict(zip(['u', 'v', 'w', 'a', 'pad'],[xyzs[:,0],xyzs[:,1],xyzs[:,2],xyzs[:,3],xyzs[:,4]])) #> assign each key to an array
xyz = {}
u = 'u'
v = 'v'
w = 'w'
a = 'a'
pad = 'pad'
for i in range(len(xyzs)):
    xyz.setdefault(u,[]).append(xyzs[i,0])
    xyz.setdefault(v,[]).append(xyzs[i,1])
    xyz.setdefault(w,[]).append(xyzs[i,2])
    xyz.setdefault(a,[]).append(xyzs[i,3])
    xyz.setdefault(pad,[]).append(xyzs[i,4])

#print(xyz)

In [90]:
print(xyz[['a']])

TypeError: unhashable type: 'list'

In [52]:
mc = MCFitter(config)
mc.process_event(xyz, cu, cv)

TypeError: unhashable type: 'list'